In [3]:
from manim import *
from manim.utils.color import Colors
# from manim_fontawesome import *
from nextgen_01_lib import *

from random import seed, shuffle
import numpy as np

Manim Community v0.17.1

## Probability via simulation

In [23]:
%%manim -v WARNING --progress_bar None NG_01_02_ProbViaSim

seed(18)

class NG_01_02_ProbViaSim(Scene):

    def construct(self):
        scale = 0.2
        POS_COLOR = BLUE
        NEG_COLOR = RED
        # SICK_SYMB = "☹︎"
        # WELL_SYMB = "☺︎"
        sick_icon = Text(SICK_SYMB).scale(5) # or ♓︎
        # sick_icon = Triangle()
        well_icon = Text(WELL_SYMB).scale(5)

        targetspace = Axes(
            y_range=[0, 1.0, 0.1],
            x_range=[0, 1.0, 0.1],
            y_length=4,
            x_length=4,
            # x_axis_config={"font_size": 36},
        )
        targetspace.shift(4*LEFT)
        self.add(targetspace)

        def as_rectangle_corners(bottom_left, top_right):
            return [
                (top_right[0], top_right[1]),
                (bottom_left[0], top_right[1]),
                (bottom_left[0], bottom_left[1]),
                (top_right[0], bottom_left[1]),
            ]
        
        def get_rectangle(bottom_left, top_right, color=POS_COLOR):
            polygon = Polygon(
                *[
                    targetspace.c2p(*i)
                    for i in as_rectangle_corners(
                        bottom_left, top_right
                    )
                ]
            )
            polygon.stroke_width = 1
            polygon.set_fill(color, opacity=0.3)
            polygon.set_stroke(color)
            return polygon

        def in_zone(zone, point):
            return (
                zone[0][0] <= point[0] < zone[1][0]
                and zone[0][1] <= point[1] < zone[1][1]
            )
        
        well_neg_zone = (0.0, 0.0), (0.9, 0.9)
        well_neg_rect = get_rectangle(*well_neg_zone, NEG_COLOR)
        self.add(well_neg_rect)
        well_pos_zone = (0.9, 0.0), (1.0, 0.9)
        well_pos_rect = get_rectangle(*well_pos_zone, POS_COLOR)
        self.add(well_pos_rect)
        sick_neg_zone = (0.0, 0.9), (0.9, 1.0)
        sick_neg_rect = get_rectangle(*sick_neg_zone, NEG_COLOR)
        self.add(sick_neg_rect)
        sick_pos_zone = (0.9, 0.9), (1.0, 1.0)
        sick_pos_rect = get_rectangle(*sick_pos_zone, POS_COLOR)
        self.add(sick_pos_rect)

        zones = [well_neg_zone, well_pos_zone, sick_neg_zone, sick_pos_zone]
        count = np.array([0, 0, 0, 0])

        def zone(point):
            for i, zone in enumerate(zones):
                if in_zone(zone, point):
                    return i, zone

        chart = BarChart(
            values=[0,0,0,0],
            bar_names=[ WELL_SYMB, WELL_SYMB, SICK_SYMB, SICK_SYMB],
            bar_colors=[NEG_COLOR, POS_COLOR, NEG_COLOR, POS_COLOR],
            y_range=[0, 1.0, 0.1],
            y_length=6,
            x_length=4,
            # x_axis_config={"font_size": 36},
        )
        chart.shift(4*RIGHT)

        ## various attempts to animate te bar labels, all doomed
        # c_bar_lbls = chart.get_bar_labels(font_size=24, 
        #     label_constructor=lambda f: DecimalNumber(float(f), num_decimal_places=2))    
        # for i, lab in enumerate(c_bar_lbls):
        #     # lab.add_updater(lambda l: l.set_value(chart.values[i]))
        #     lab.add_updater(lambda l: l.become(chart.bar_labels[i]))
        # self.add(chart, c_bar_lbls)
        # c_bar_lbls.add_updater(lambda l: l.become(chart.bar_labels))

        for n_points_now, dur in [(1,1.0),] *10 + [(2*k , 1/k) for k in range(1, 25)]: 
            for step in range(n_points_now):
                # override first few points so the bars start out even-ish
                if count.sum() == 0:
                    p = np.array([0.45, 0.95])
                elif count.sum() == 1:
                    p = np.array([0.12, 0.8])
                elif count.sum() == 2:
                    p = np.array([0.91, 0.97])
                else:
                    p = np.random.random(2)
                self.add(
                    Dot(
                        targetspace.c2p(*p)
                    ))
                i, z = zone(p)
                count[i] += 1
                self.play(
                    chart.animate.change_bar_values(count/count.sum()),
                    # Transform(
                    #     c_bar_lbls,
                    #     chart.bar_labels),
                    run_time=dur)




In [65]:
%%manim -v WARNING --progress_bar None WalkingThroughDoor
from nextgen_01_defs import *
from nextgen_01_lib import *

class WalkingThroughDoor(Scene):
    def construct(self):
        person = SVGMobject(
            "noun-person-1492700.svg",
            fill_color=RED,
        ).set_z_index(0)
        
        corridor = Rectangle(
            height=2, width=2,
            color=BLACK,
            fill_color=BLACK,
            fill_opacity=1.0
        ).shift(LEFT*1.5).set_z_index(1)

        door = Rectangle(height=2, width=1).set_z_index(3)

        self.add(corridor)
        self.play(
            FadeIn(door),
            FadeIn(person)
        )
        self.play(MoveAlongPath(person, Line(LEFT*1.5, RIGHT*1.5)), run_time=2)
